# D&D encounter manager 

### Requirements
- Select enemies for an encounter if provided with a challenge rating and environment (I can upload possible options and it should randomly choose from a list of say 100)
- Roll 20-sided die and determine order in which players will take turns
- Quick access to monster stats and possible attacks/moves
- Manage HP for each monster and track turn count-downs where applicable
- Extra: LLM API to help generate interesting names, monster characteristics / traits, things they say, things they are wearing

In [1]:
import numpy as np
import random
import math
from operator import attrgetter
#from collections import Counter
#from itertools import repeat

## Assignment 1

1. Create your own python [function](https://www.geeksforgeeks.org/python-functions/) that returns a random number from 1-20. You should use pythons [randint](https://www.w3schools.com/python/ref_random_randint.asp) method to help you.
2. Make a [list](https://www.w3schools.com/python/python_lists.asp) of names (make up like 5 different names).
3. Write a for [loop](https://www.w3schools.com/python/python_for_loops.asp) that goes through your list and for each name:
* A) prints the name;
* B) prints a random number 1-20 using your function from (1).

In [2]:
def roll_dice(sides = 20):
    """Returns a number between 1 and the sides of a dice using a random uniform distribution (default 20-sided)"""
    roll = np.random.randint(1, sides + 1)
    
    if sides not in {4, 6, 8, 10, 12, 20, 100}:
        print(f"Warning: It is uncommon to roll a {sides}-sided die. Are you sure?") # Can also use warnings.warn
        return roll
    else:
        return roll


Nice commenting on functions!

nit: roll_dice_20 is returning a double here. We probably want to represent a dice roll as an int. This can be done by casting roll into an int: `roll = int(roll)` or by using a different random generating function. I personally would just use randint with something like:
```
from random import randint

roll = randint(1, 21)
```

<font color='hotpink'>
    
- Good idea generalizing your "roll_dice_20" function. I think that a better naming for this function would be just "roll_dice." The argument name "sides" conveys to the caller that the number of sides can be chosen (no need for word "any").
    
- for the line `if sides not in [4, 6, 8, 10, 12, 20, 100]`, it is checking membership in a list. Better to use a `set`: `if sides not in {4, 6, 8, 10, 12, 20, 100}`. When Python checks for membership of `x` in a list, it needs to iterate over every element of the list to see if it equals `x`. So this takes a **really** long time for big lists. For sets, it is a **hash map** so it can check for membership of `x` in constant time. This means, *no matter how big your set is, it always takes the same amount of time to check if something is in it.* This is an amazing property of hash maps.

- Your warning isn't printing as it comes after the return statement. So the function never gets to it! You should just put the warning before the return statement.
</font> 

In [3]:
roll_dice(15)

2

In [4]:
combatants = ["unnamed_goblin_1", "unnamed_goblin_2", "Ivan", "unnamed_goblin_4", "goblin_lord_1"]

In [5]:
print(combatants[0])

unnamed_goblin_1


In [6]:
for n in combatants:
    print(n)
    print(roll_dice())

unnamed_goblin_1
7
unnamed_goblin_2
8
Ivan
10
unnamed_goblin_4
16
goblin_lord_1
14


In [7]:
for n in combatants:
    print(f"Combatant: {n}, roll: {roll_dice()}")

Combatant: unnamed_goblin_1, roll: 2
Combatant: unnamed_goblin_2, roll: 9
Combatant: Ivan, roll: 15
Combatant: unnamed_goblin_4, roll: 16
Combatant: goblin_lord_1, roll: 19


Cool fact, in python you can use f-strings to easily print information about variables. You could do something like
```
print(f"Combatant: {n}, roll: {roll_dice_20()}")
```

## Assignment 2

1. Just to practice dictionaries (we will do something fancier in a later assignment to manage player stats), create a [dictionary](https://www.w3schools.com/python/python_dictionaries.asp) whose keys are your combatants and whose values are their dice rolls.

2. We want to meet the requirement to decide the order players take turns. To programmatically break ties of a 20-sides dice, is a little tricky. Instead, let's just take your combatants list and apply a random permutation, selected uniformly from all possible permuations. By symmetry (no player is prefered in either method), this has the same probabability distribution of as ordering players with a 20-sided die strategy. Use the [shuffle function](https://pynative.com/python-random-shuffle/) to randomly order your list of combatants.

3. Create a `Goblin` [class](https://www.w3schools.com/python/python_classes.asp). Your goblin should be initialized with a fixed hp that all goblins have (i.e., initialize `self.hp`). Goblins should also have other properties associated to them. I don't know what those should be, but I'll let you come up with them. Your class should have methods for actions a goblin can take, like attack maybe? I will let you figure those out lol. You can leave the method implementations blank for now. Focus on learning how you can define different variables and methods in a class from the linked reading. 

In [8]:
len(combatants)

5

In [9]:
combatant_rolls = {}

for i in range(len(combatants)):
    combatant_rolls.update({combatants[i]: roll_dice()})

print(combatant_rolls)

{'unnamed_goblin_1': 13, 'unnamed_goblin_2': 7, 'Ivan': 7, 'unnamed_goblin_4': 20, 'goblin_lord_1': 16}


<font color="hotpink">
This for loop is a great opportunity to drop your accent. Let's look at better ways to do this:

- Instead of doing this `combatant_rolls.update({combatants[i]: roll_dice_20()})`, you just need to do this: ```combatant_rolls[combatants[i]] = roll_dice20()```

- But why iterate over range? Much better is:

```
for combatant in combatants:
    combatant_rolls[combatant] = roll_dice_20()
```

Note how descriptive python is. Very sexy.

- Let's be fancy girls. You can replace your whole for loop with a dictionary comprehension:
  ```
  combatant_rolls = {combatant: roll_dice_20() for combatant in combatants}
  ```

</font>

In [10]:
combatant_rolls = {combatant: roll_dice() for combatant in combatants}

print(combatant_rolls)

{'unnamed_goblin_1': 3, 'unnamed_goblin_2': 17, 'Ivan': 9, 'unnamed_goblin_4': 11, 'goblin_lord_1': 13}


In [11]:
random.shuffle(combatants) # Shuffles in-place

print(combatants)

['unnamed_goblin_2', 'goblin_lord_1', 'unnamed_goblin_1', 'Ivan', 'unnamed_goblin_4']


In [12]:
def num_to_ordinal(n):
    """Convert a number to its ordinal representation"""
    #mod = n % 10
    if n % 10 in range(4, 10):
        suffix = "th"
    elif n % 10 == 0:
        suffix = "th"
    elif n % 10 == 1:
        suffix = "st"
    elif n % 10 == 2:
        suffix = "nd"
    elif n % 10 == 3:
        suffix = "rd"
    return f"{n}{suffix}"

## 10/19

Consider what happens here when n = 11

In [13]:
def ability_score_to_modifier(ability_score):
    """Converts ability scores to ability score modifiers"""
    modifier = math.floor((ability_score - 10) / 2)
    
    return modifier

def monster_attack(target, weapon, attack_roll_modifier, damage_roll_modifier):
    attack_roll = roll_dice(20)
    
    damage_dice_rolls = []
    for rolls in range(weapon.damageDiceCount):
        damage_dice_rolls.append(roll_dice(weapon.damageDiceSides))
        rolls += 1
    
    attack_details = Attack(
        target = target,
        attackRoll = attack_roll,
        attackRollModified = attack_roll + attack_roll_modifier,
        damageRoll = sum(damage_dice_rolls) + damage_roll_modifier
        )
    
    return attack_details

def player_attack(player, target, weapon):
    attack_roll = roll_dice(20)
    
    damage_dice_rolls = []
    for rolls in range(weapon.damageDiceCount):
        damage_dice_rolls.append(roll_dice(weapon.damageDiceSides))
        rolls += 1

    if weapon.weaponType == "melee":
        attack_and_damage_roll_modifier = ability_score_to_modifier(player.abilityScores.strength)
    elif weapon.weaponType == "ranged":
        attack_and_damage_roll_modifier = ability_score_to_modifier(player.abilityScores.dexterity)
    
    attack_details = Attack(
        target = target,
        attackRoll = attack_roll,
        attackRollModified = attack_roll + player.proficiencyBonus + attack_and_damage_roll_modifier,
        damageRoll = sum(damage_dice_rolls) + attack_and_damage_roll_modifier
        )
    
    return attack_details

## 10/19

I think there may be a misunderstanding or typo here:

```
for rolls in range(weapon.damageDiceCount):
        damage_dice_rolls.append(roll_dice(weapon.damageDiceSides))
        rolls += 1
```

You want to loop over the total number of weapon.damageDiceCount right? There is no need to increment `rolls`. In other words, you can remove the line `rolls += 1`. Also, I would call it `roll` instead of `rolls`. You are looping through each `roll` in that range. 

---


I see that this code is repeated twice: 
```
damage_dice_rolls = []
    for rolls in range(weapon.damageDiceCount):
        damage_dice_rolls.append(roll_dice(weapon.damageDiceSides))
```
I know it's a small repetition, but you may want to make it into a function: `def get_damage_dice_rolls(total_rolls)`

---

With this code:
```
    if weapon.weaponType == "melee":
        attack_and_damage_roll_modifier = ability_score_to_modifier(player.abilityScores.strength)
    elif weapon.weaponType == "ranged":
        attack_and_damage_roll_modifier = ability_score_to_modifier(player.abilityScores.dexterity)
```
There is a chance that `attack_and_damage_roll_modifier` never gets defined, if weaponType does not match one of those two choices. Consider what to do in this case. You can do this in an `else` block. This error could happen if you add a new weaponType in the future and forget to modify the player_attack function.

In [14]:
class AbilityScores:
    """Ability scores for creatures"""
    
    def __init__(self, strength, dexterity, constitution, intelligence, wisdom, charisma):
        ability_scores_set = {strength, dexterity, constitution, intelligence, wisdom, charisma}

        if any(score > 30 for score in ability_scores_set):
            print("Ability scores must be between 1 and 30.")
        elif any(score < 1 for score in ability_scores_set):
            print("Ability scores must be between 1 and 30.")
        else:
            self.strength = strength
            self.dexterity = dexterity
            self.constitution = constitution
            self.intelligence = intelligence
            self.wisdom = wisdom
            self.charisma = charisma

class Alignment:
    """Alignment for creatures"""

    def __init__(self, lawfulness, morality):
        self.valid_lawfulness = {"lawful", "neutral", "chaotic"}
        self.valid_morality = {"good", "neutral", "evil"}

        if lawfulness in self.valid_lawfulness:
            self.lawfulness = lawfulness
        else:
            print(f"Invalid lawfulness: {lawfulness}. Must be one of {self.valid_lawfulness}.")

        if morality in self.valid_morality:
            self.morality = morality
        else:
            print(f"Invalid morality: {morality}. Must be one of {self.valid_morality}.")

class Attack:
    """Attacks for creatures"""

    def __init__(self, target, attackRoll, attackRollModified, damageRoll):
        self.target = target
        self.attackRoll = attackRoll
        self.attackRollModified = attackRollModified
        self.damageRoll = damageRoll

## 10/19

Nice checking for edge cases in `AbilityScores` and in `Alignment`. Maybe later we can talk about different ways to handle these, like throwing exceptions. This will ensure the code doesn't run and create a half-assed `Alignment` object. I don't think it is a critical thing to do though.

In [15]:
# Weapon classes
class Scimitar:
    """Weapon: Scimitar (2014 Player's Handbook)"""

    def __init__(self):
        self.name = "scimitar"
        self.cost = 25
        self.damageDiceCount = 1
        self.damageDiceSides = 6
        self.weight = 3
        self.weaponType = "melee"

class BattleAxe:
    """Weapon: Battle Axe (2014 Player's Handbook)"""

    def __init__(self):
        self.name = "battle axe"
        self.cost = 10
        self.damageDiceCount = 1
        self.damageDiceSides = 8
        self.weight = 4
        self.weaponType = "melee"

In [16]:
# Monster classes
class Goblin:
    """Goblin creature (2014 Monster Manual)"""

    def __init__(self, name):
        self.name = name
        self.size = "small"
        self.creatureType = "humanoid"
        self.alignment = Alignment("neutral", "evil")
        self.armorClass = 15
        self.hitPointsMax = 7
        self.hitPoints = self.hitPointsMax
        self.speed = 30
        self.weapons = [Scimitar()]
        self.abilityScores =  AbilityScores(
            strength = 8,
            dexterity = 14,
            constitution = 10,
            intelligence = 10,
            wisdom = 8,
            charisma = 8
            )
        self.proficiencyBonus = 2
        self.challengeRating = 0.25
        self.initiativeRoll = None
        self.initiativeRollModified = None
        self.initiativeRank = None
    
    def __str__(self):
        return f"Goblin {self.name} smells blood!"

    def rollForInitiative(self):
        self.initiativeRoll = roll_dice(20)
        self.initiativeRollModified = self.initiativeRoll + ability_score_to_modifier(self.abilityScores.dexterity)
        return f"Goblin {self.name} rolled a {self.initiativeRoll}."

    def attack(self, target, weapon):
        attackDetails = monster_attack(
            target = target,
            weapon = weapon,
            attack_roll_modifier = 4,
            damage_roll_modifier = 2
            )
        return attackDetails

In [17]:
class Player:
    """Playable hero"""

    def __init__(self,
                 name,
                 race,
                 playerClass,
                 abilityScores,
                 proficiencyBonus,
                 alignment,
                 hitPointsMax,
                 armorClass,
                 weapons
                ):
        self.name = name
        self.race = race
        self.playerClass = playerClass
        self.abilityScores = abilityScores
        self.proficiencyBonus = proficiencyBonus
        self.alignment = alignment
        self.hitPointsMax = hitPointsMax
        self.hitPoints = self.hitPointsMax
        self.armorClass = armorClass
        self.weapons = weapons
        self.size = "medium"
        self.speed = 30
        self.initiativeRoll = None
        self.initiativeRollModified = None
        self.initiativeRank = None

    def __str__(self):
        return f"{self.playerClass.capitalize()} {self.name} jumps into battle!"

    def rollForInitiative(self):
        self.initiativeRoll = roll_dice(20)
        self.initiativeRollModified = self.initiativeRoll + ability_score_to_modifier(self.abilityScores.dexterity)
        return f"{self.name} rolled a {self.initiativeRoll}."

    def attack(self, target, weapon):
        attackDetails = player_attack(
            player = self,
            target = target,
            weapon = weapon
            )
        return attackDetails

<font color="hotpink">

- Whoa! Awesome use of `__str__`. This is one of python's **magic methods**
- Regarding: `    stats = [8, 14, 10, 10, 8, 8] # Strength, dexterity, constitution, intelligence, wisdom, charisma
` It is not good to pack things in a list where there is hidden meaning. It relies on someone reading that comment. A much better thing to do, is create a `Stats` class, and make "Strength, dexterity, constitution, intelligence, wisdom, charisma" each a property of the class. Your goblin can then have a property `self.stats = Stats(strength = 5, dexterity = 6, ....)`. The Stats can then be used by other monsters as well and is a common interface understood between them.
- So, for any class in Python (and other OO languages), there are **instances** (or **objects**) of that class. For example, `Goblin` is your class, and `goblin1 = Goblin('Joe')` is an instance of it. All the `self` properties like `self.name` above belong to the instance. For example, goblin1 may be named Joe while goblin2 may be named "h. solo".

Many of the other variables you are defining belong to the class currently, and they can't be changed for individual instances. This is probably incorrect, as you will want the hp to be a property of each instance (goblin1 may be down to 5 hp while h. solo has 6 hp.
- Most methods you write should be made to manipulate your instance's variables. This is usually done by something like
  `def attack(self, type = "melee")`
- On the other hand, `class methods` are bound to the class and not individual instances of the class. This means, they cannot manipulate variables belonging to a goblin1. I would say, class methods are much less used (although they are definitely useful).
- You are packing wayyy too much in the return type of a method. Instead, lets record that information in properties of your object and in other classes. You do not want the caller of your method to have to know what each item in the returned list should mean. See next assignment.

</font>

## 10/18


Kinda either super racist or super woke to give players a race. Have a story around this ready for HR.

---
Why not make the `player_attack` function a method in `Player`?

## Assignment 3

1. Modify your attack method above to:
 -  not be a class method
 -  I am assuming "damage_roll" determines how much hp is lost. Modify the attack method to decrease your goblin instance's self.hp by the appropriate amount
 -  The short_range and long_range variables never change. Make these properties of your class. You can then have another function that returns these: `get_short_range()`, `get_long_range()`.
 -  The attack method should just return the attack roll result.
2. Make the Stats class mentioned above
3. Add more methods to your Goblin class.
4. Create two other monster classes. Comment what methods/properties **all** monster classes should have.
5. Create a `Player` class and its properties and methods. Create an instance of the Player class `Ivan`. This player's stats should be superior to all other players.


In [18]:
goblin1 = Goblin("Joe")

In [19]:
player1 = Player(name = "Ivan",
                 race = "human",
                 playerClass = "barbarian",
                 abilityScores = AbilityScores(16, 15, 14, 13, 12, 11),
                 proficiencyBonus = 3,
                 alignment = Alignment("lawful", "good"),
                 hitPointsMax = 30,
                 armorClass = 15,
                 weapons = [BattleAxe()]
                )

## 10/19

Please add "sexy" to alignment here. And race should be "latinx".

## 10/19

Lets create a `GameHandler` class. All the logic below this comment should be implemented in that class.

In [20]:
combatants = [goblin1, player1]

random.shuffle(combatants) # Introduce combatants in a random order

for combatant in combatants:
    print(combatant)

Goblin Joe smells blood!
Barbarian Ivan jumps into battle!


In [21]:
print("Roll for initiative!")
display(goblin1.rollForInitiative())
display(player1.rollForInitiative())

Roll for initiative!


'Goblin Joe rolled a 15.'

'Ivan rolled a 12.'

In [22]:
# Sort combatants list by modified initiative roll
combatants.sort(key = attrgetter('initiativeRollModified'), reverse = True) # Going to assume no ties for now, need to implement tie resolver later

# Assign list index as initiativeRank attribute to lock in order of combat
index = 0
for combatant in combatants:
    combatant.initiativeRank = index
    print(f'{combatant.name} will take turn {num_to_ordinal(index + 1)}.')
    index += 1

Joe will take turn 1st.
Ivan will take turn 2nd.


In [23]:
active_player = combatants[0] # First index will attack
attack1 = active_player.attack(target = combatants[1], weapon = active_player.weapons[0]) # Targets the only other combatant with the default weapon

print(f'For troubleshooting only - attack roll: {attack1.attackRoll}')
print(f'For troubleshooting only - modified attack roll: {attack1.attackRollModified}')
print(f'For troubleshooting only - damage roll: {attack1.damageRoll}')
print(f'For troubleshooting only - armor class: {combatants[1].armorClass}')

For troubleshooting only - attack roll: 20
For troubleshooting only - modified attack roll: 24
For troubleshooting only - damage roll: 3
For troubleshooting only - armor class: 15


In [24]:
# Show target hit points before attack
print(attack1.target.hitPoints)

30


## 10/19

You may want to consider putting the logic below this comment into your attack function itself.

In [30]:
if attack1.attackRoll == 1:
    print("Womp womp it's a nat 1 and misses.")
elif attack1.attackRoll == 20:
    attack1.target.hitPoints = attack1.target.hitPoints - (attack1.damageRoll * 2)
    print(f'A critical hit dealing {attack1.damageRoll * 2} damage!')
elif attack1.attackRollModified >= attack1.target.aArmorClass:
    attack1.target.hitPoints = attack1.target.hitPoints - attack1.damageRoll
    print(f'It hits, causing {attack1.damageRoll} damage.')
elif attack1.attackRollModified < attack1.target.armorClass:
    print("It misses.")

A critical hit dealing 6 damage!


In [32]:
# Show target hit points after attack
print(attack1.target.hitPoints)
print(combatants[1].hitPoints) # This does the same thing! Cool!

6
6


In [35]:
if attack1.target.hitPoints <= 0:
    print(f'{attack1.target.name} faints.')
else:
    print(f'{attack1.target.name} is still standing.')

Ivan is still standing.


## 10/19 Assignment 4

Address comments above, including creating a `GameHandler` class. Read up on 
Python inheritance [here](https://www.w3schools.com/python/python_inheritance.asp) and whatever other sources you find useful. Create a 
`Monster` parent class that all monster inherit from. Move your `monster_attack` function to the `Monster` parent class. Make a `Weapon` class 
and have your different weapons like `BattleAxe` and `Dildo` inherit from it. This should reduce the repetitiveness in code for the different weapons you are creating (they all have the exact same variables but are defined with different values).

In [26]:
# Show target hit points after attack
print(attack1.target.hitPoints)
print(combatants[1].hitPoints) # This does the same thing! Cool!

24
24


In [27]:
if attack1.target.hitPoints <= 0:
    print(f'{attack1.target.name} faints.')
else:
    print(f'{attack1.target.name} is still standing.')

Ivan is still standing.
